# Generate heatmaps to assess any well-level differences

For heatmaps with clustering (all the plots split by heart and one for the whole plate), we use Pearson's correlation.

In [1]:
# Libraries (suppress startup messages and warnings)
pkgs <- c("arrow", "dplyr", "tidyr", "RColorBrewer", "grid", "ComplexHeatmap", "circlize", "magrittr")
invisible(lapply(pkgs, function(p) {
  suppressPackageStartupMessages(
    suppressWarnings(
      library(p, character.only = TRUE, quietly = TRUE, warn.conflicts = FALSE)
    )
  )
}))

In [2]:
# Set plate to process: "redo" or "original"
plate_to_process <- "redo"

# Set output directory based on plate_to_process
if(plate_to_process == "redo") {
  output_dir <- file.path(".", "figures", "heatmaps", "redo_DMSO_plate")
} else if(plate_to_process == "original") {
  output_dir <- file.path(".", "figures", "heatmaps", "original_DMSO_plate")
} else {
  stop("Unknown plate_to_process value")
}

# Create directory if it doesn't exist
if(!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = TRUE)
}

# Build resolved base directory (one level up from notebook working dir)
base_dir <- normalizePath(file.path(getwd(), ".."))
sc_dir <- file.path(base_dir, "3.preprocessing_profiles", "data", "single_cell_profiles")

fname <- switch(
  plate_to_process,
  redo = "CARD-CelIns-CX7_251110170001_sc_feature_selected.parquet",
  original = "CARD-CelIns-CX7_251023130003_sc_feature_selected.parquet",
  stop("Unknown plate_to_process: ", plate_to_process)
)

data_path <- tryCatch(normalizePath(file.path(sc_dir, fname), mustWork = TRUE),
                      error = function(e) stop(sprintf("Could not find parquet %s in %s: %s", fname, sc_dir, e$message)))

# Read single-cell parquet
single_cell_data <- arrow::read_parquet(data_path)
print(sprintf("Loaded single-cell data from: %s", data_path))
dim(single_cell_data)
head(single_cell_data)

[1] "Loaded single-cell data from: /home/jenna/predicting_cardiac_fibrosis_etiologies/3.preprocessing_profiles/data/single_cell_profiles/CARD-CelIns-CX7_251110170001_sc_feature_selected.parquet"


[1] 8852 1030

Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_cell_type,Metadata_heart_failure_type,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,⋯,Nuclei_Texture_InfoMeas2_PM_3_02_256,Nuclei_Texture_InfoMeas2_PM_3_03_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_00_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_02_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_00_256,Nuclei_Texture_SumVariance_ER_3_01_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_PM_3_01_256
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
B,2,2,DMSO,Healthy,NA,658.9502,133.5264,655.8408,145.9062,⋯,1.5036264,0.57339617,0.6820916,-1.19241337,-0.97234582,0.08841673,0.1811709,-0.2270292,0.9458709,-0.1761020
B,2,2,DMSO,Healthy,NA,714.7930,165.7729,650.9471,167.0519,⋯,0.2968638,0.04597522,0.2168721,0.15111905,0.24937612,-0.13487506,-0.3223056,-0.1429148,-0.3980370,-0.1191326
B,2,2,DMSO,Healthy,NA,652.6875,189.7704,619.4071,199.3478,⋯,0.1154466,-0.10138457,0.1739141,-0.25919729,-0.08466076,-0.12207075,-0.2451825,-0.2739575,-0.4054884,-0.1413972
B,2,2,DMSO,Healthy,NA,888.7179,208.9546,871.2041,158.0898,⋯,1.1839115,0.96668892,-2.4483637,-2.02382580,-1.88979559,-1.00722101,1.0671333,-0.2542584,0.6362578,0.2465465
B,2,2,DMSO,Healthy,NA,599.6122,235.4967,568.7633,236.2237,⋯,0.2951993,0.76309198,-0.9513179,0.04254455,-2.38501610,-0.17391894,-0.1940491,-0.2169824,0.6297157,-0.0335783
B,2,2,DMSO,Healthy,NA,601.6549,212.2925,649.7332,210.1268,⋯,0.3288961,0.17403247,0.2897362,-0.29109014,-1.39358476,0.69656168,-0.3025933,-0.2682725,-0.1030006,-0.1990257


In [3]:
# Aggregate to well-level using median
aggregate_df <- single_cell_data %>%
  group_by(Metadata_Plate, Metadata_Well, Metadata_heart_number, Metadata_treatment, Metadata_cell_type) %>%
  summarize(across(where(is.numeric), ~median(.x, na.rm = TRUE)), .groups = "drop")

print(sprintf("Aggregated well-level data shape: %d x %d", nrow(aggregate_df), ncol(aggregate_df)))
head(aggregate_df)

[1] "Aggregated well-level data shape: 56 x 1017"


Metadata_Plate,Metadata_Well,Metadata_heart_number,Metadata_treatment,Metadata_cell_type,Metadata_WellCol,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,⋯,Nuclei_Texture_InfoMeas2_PM_3_02_256,Nuclei_Texture_InfoMeas2_PM_3_03_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_00_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_02_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_00_256,Nuclei_Texture_SumVariance_ER_3_01_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_PM_3_01_256
<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CARD-CelIns-CX7_251110170001,B02,2,DMSO,Healthy,2,585.7139,552.5414,574.7386,547.9288,⋯,0.13782300,0.10084696,-0.074906594,-0.35735195,-0.35932041,-0.4311501,-0.1805315,-0.1499531,-0.1598620,-0.096577483
CARD-CelIns-CX7_251110170001,B03,7,DMSO,Healthy,3,509.8476,543.3492,511.9647,535.2808,⋯,0.05513087,0.10861792,0.056649600,0.22513752,0.21317741,-0.7055042,-0.2896356,-0.1684752,-0.3413805,-0.057370679
CARD-CelIns-CX7_251110170001,B04,2,None,Healthy,4,519.3087,515.1040,504.6346,501.7067,⋯,-0.05255958,0.04188301,-0.003849796,-0.06914475,-0.06493529,-0.6572168,-0.2515078,-0.1280578,-0.2805742,-0.070661409
CARD-CelIns-CX7_251110170001,B05,23,DMSO,Failing,5,514.0659,529.1443,498.4657,550.1612,⋯,0.19485241,0.11439366,0.182120887,0.38894712,0.41188483,-0.9728520,-0.3124686,-0.2385582,-0.3765594,0.015323976
CARD-CelIns-CX7_251110170001,B06,25,DMSO,Failing,6,585.2499,532.8860,566.6197,517.1669,⋯,0.05815490,0.09155244,0.144923920,0.69424947,0.67395473,-1.0082106,-0.2548120,-0.2217183,-0.4166317,0.008444507
CARD-CelIns-CX7_251110170001,B07,46,DMSO,Failing,7,514.7869,542.8452,524.4035,539.8119,⋯,0.07909100,0.03607315,0.349707233,0.65147843,0.66084581,-1.0735312,-0.3450777,-0.2199232,-0.4198590,0.061722835


In [4]:
# Loop over heart numbers
for (heart_num in unique(aggregate_df$Metadata_heart_number)) {
  
  heart_df <- aggregate_df %>% filter(Metadata_heart_number == heart_num)
  if (nrow(heart_df) == 0) {
    message(sprintf("No rows found for Heart #%s, skipping...", heart_num))
    next
  }
  
  # Loop over treatments
  for (treatment in unique(heart_df$Metadata_treatment)) {
    
    df <- heart_df %>% filter(Metadata_treatment == treatment)
    if (nrow(df) == 0) next
    
    # Separate metadata and feature columns
    metadata_cols <- names(df)[grepl('^Metadata_', names(df))]
    feature_cols <- setdiff(names(df), metadata_cols)
    
    # Use all features
    mat <- df[feature_cols]
    mat[!is.finite(as.matrix(mat))] <- 0
    mat_z <- scale(as.matrix(mat), center = TRUE, scale = TRUE)
    mat_z[is.na(mat_z)] <- 0
    
    # Row labels are just wells
    row_labels <- df$Metadata_Well
    rownames(mat_z) <- row_labels
    
    # Row annotation for wells
    row_ann_df <- data.frame(Well = as.character(df$Metadata_Well))
    rownames(row_ann_df) <- row_labels
    
    # Colors for wells
    wells <- unique(row_ann_df$Well)
    num_wells <- length(wells)
    palette_base <- colorRampPalette(RColorBrewer::brewer.pal(min(12, max(3, num_wells)), "Set3"))(num_wells)
    annotation_colors <- list(Well = setNames(palette_base, wells))
    
    row_ha <- rowAnnotation(
      Well = row_ann_df$Well,
      col = annotation_colors,
      show_annotation_name = TRUE
    )
    
    # Define correlation-based distance function
    dist_cor <- function(x) {
      as.dist(1 - cor(t(x), use = "pairwise.complete.obs"))
    }
    
    # Correlation-based clustering
    hc_rows <- hclust(dist_cor(mat_z), method = "average")
    hc_cols <- hclust(dist_cor(t(mat_z)), method = "average")
    
    # Heatmap
    ht <- Heatmap(
      mat_z,
      name = "Z-score",
      show_row_names = FALSE,
      show_column_names = FALSE,
      cluster_rows = hc_rows,
      cluster_columns = hc_cols,
      col = colorRamp2(
        breaks = c(min(mat_z), 0, max(mat_z)),
        colors = c("#ca0020", "white", "#0571b0")  # red → white → blue
      ),
      heatmap_legend_param = list(title = "Z-score"),
      column_title = sprintf(
        "Heart #%s: Well-level feature heatmap (n = %d)",
        heart_num, length(feature_cols)
      ),
      column_title_gp = gpar(fontsize = 14, fontface = "bold")
    )
    
    # Save to file in output_dir (sanitize treatment for filename)
    safe_treatment <- gsub("[^A-Za-z0-9_\\-]", "_", as.character(treatment))
    output_file <- file.path(
      output_dir,
      sprintf("heart_%s_%s_well_level_heatmap.png", heart_num, safe_treatment)
    )
    png(output_file, width = 2800, height = 2000, res = 400)
    draw(ht + row_ha, heatmap_legend_side = "right", annotation_legend_side = "right")
    dev.off()
    
    message(sprintf(
      "Saved heatmap for Heart #%s, Treatment '%s' (%d features) to %s",
      heart_num, treatment, length(feature_cols), output_file
    ))
  }
}

Saved heatmap for Heart #2, Treatment 'DMSO' (986 features) to ./figures/heatmaps/redo_DMSO_plate/heart_2_DMSO_well_level_heatmap.png

Saved heatmap for Heart #2, Treatment 'None' (986 features) to ./figures/heatmaps/redo_DMSO_plate/heart_2_None_well_level_heatmap.png

Saved heatmap for Heart #7, Treatment 'DMSO' (986 features) to ./figures/heatmaps/redo_DMSO_plate/heart_7_DMSO_well_level_heatmap.png

Saved heatmap for Heart #23, Treatment 'DMSO' (986 features) to ./figures/heatmaps/redo_DMSO_plate/heart_23_DMSO_well_level_heatmap.png

Saved heatmap for Heart #25, Treatment 'DMSO' (986 features) to ./figures/heatmaps/redo_DMSO_plate/heart_25_DMSO_well_level_heatmap.png

Saved heatmap for Heart #46, Treatment 'DMSO' (986 features) to ./figures/heatmaps/redo_DMSO_plate/heart_46_DMSO_well_level_heatmap.png

Saved heatmap for Heart #47, Treatment 'DMSO' (986 features) to ./figures/heatmaps/redo_DMSO_plate/heart_47_DMSO_well_level_heatmap.png



In [5]:
# Identify metadata vs feature columns
metadata_cols <- names(aggregate_df)[grepl('^Metadata_', names(aggregate_df))]
feature_cols <- setdiff(names(aggregate_df), metadata_cols)

# Use all features
mat <- as.data.frame(aggregate_df[feature_cols], stringsAsFactors = FALSE)
mat[!is.finite(as.matrix(mat))] <- 0

# Z-score each column (feature)
mat_z <- scale(as.matrix(mat), center = TRUE, scale = TRUE)
mat_z[is.na(mat_z)] <- 0

# Row labels
row_labels <- paste0("H", aggregate_df$Metadata_heart_number, "_", seq_len(nrow(aggregate_df)))
rownames(mat_z) <- row_labels

# Row annotation data.frame
row_ann_df <- data.frame(
  Metadata_heart_number = as.character(aggregate_df$Metadata_heart_number),
  Metadata_cell_type = as.character(aggregate_df$Metadata_cell_type),
  Metadata_treatment = as.character(aggregate_df$Metadata_treatment),
  stringsAsFactors = FALSE
)
rownames(row_ann_df) <- row_labels

# Helper for generating palettes
get_palette <- function(n, brewer_name, brewer_max) {
  if (n <= 0) return(character(0))
  if (n <= brewer_max) return(RColorBrewer::brewer.pal(max(3, n), brewer_name)[seq_len(n)])
  colorRampPalette(RColorBrewer::brewer.pal(brewer_max, brewer_name))(n)
}

# Create color mappings
heart_lv <- sort(unique(row_ann_df$Metadata_heart_number))
celltype_lv <- sort(unique(row_ann_df$Metadata_cell_type))
treatment_lv <- sort(unique(row_ann_df$Metadata_treatment))

heart_colors <- setNames(get_palette(length(heart_lv), "Set3", 12), heart_lv)
celltype_colors <- setNames(get_palette(length(celltype_lv), "Dark2", 8), celltype_lv)
treatment_colors <- setNames(get_palette(length(treatment_lv), "Paired", 12), treatment_lv)

# Turn annotation columns into factors
row_ann_df$Metadata_heart_number <- factor(row_ann_df$Metadata_heart_number, levels = names(heart_colors))
row_ann_df$Metadata_cell_type <- factor(row_ann_df$Metadata_cell_type, levels = names(celltype_colors))
row_ann_df$Metadata_treatment <- factor(row_ann_df$Metadata_treatment, levels = names(treatment_colors))

# Row annotation object
row_ha <- rowAnnotation(
  Heart = row_ann_df$Metadata_heart_number,
  CellType = row_ann_df$Metadata_cell_type,
  Treatment = row_ann_df$Metadata_treatment,
  col = list(
    Heart = heart_colors,
    CellType = celltype_colors,
    Treatment = treatment_colors
  ),
  show_annotation_name = TRUE,
  annotation_name_gp = gpar(fontsize = 12)
)

# Define correlation-based distance function
dist_cor <- function(x) {
  cor_mat <- cor(t(x), use = "pairwise.complete.obs")
  as.dist(1 - cor_mat)
}

# Hierarchical clustering for rows and columns
hc_rows <- hclust(dist_cor(mat_z), method = "average")
hc_cols <- hclust(dist_cor(t(mat_z)), method = "average")

# --- Heatmap with clustering ---
ht_clustered <- Heatmap(
  mat_z,
  name = "Z-score",
  show_row_names = FALSE,
  show_column_names = FALSE,
  cluster_rows = as.dendrogram(hc_rows),
  cluster_columns = as.dendrogram(hc_cols),
  col = colorRamp2(
    breaks = c(min(mat_z, na.rm = TRUE), 0, max(mat_z, na.rm = TRUE)),
    colors = c("#ca0020", "white", "#0571b0")
  ),
  heatmap_legend_param = list(title = "Z-score"),
  right_annotation = row_ha,
  column_title = sprintf("All Hearts: Feature Heatmap (Clustered, n = %d)", length(feature_cols)),
  column_title_gp = gpar(fontsize = 16, fontface = "bold")
)

# --- Heatmap without clustering ---
ht_unclustered <- Heatmap(
  mat_z,
  name = "Z-score",
  show_row_names = FALSE,
  show_column_names = FALSE,
  cluster_rows = FALSE,
  cluster_columns = FALSE,
  col = colorRamp2(
    breaks = c(min(mat_z, na.rm = TRUE), 0, max(mat_z, na.rm = TRUE)),
    colors = c("#ca0020", "white", "#0571b0")
  ),
  heatmap_legend_param = list(title = "Z-score"),
  right_annotation = row_ha,
  column_title = sprintf("All Hearts: Feature Heatmap (Unclustered, n = %d)", length(feature_cols)),
  column_title_gp = gpar(fontsize = 16, fontface = "bold")
)

# Ensure output directory exists
if (!dir.exists(output_dir)) dir.create(output_dir, recursive = TRUE)

# Save clustered version
output_file_clustered <- file.path(output_dir, "all_hearts_clustered_heatmap.png")
png(filename = output_file_clustered, width = 5500, height = 4000, res = 400)
draw(ht_clustered, heatmap_legend_side = "right", annotation_legend_side = "right", merge_legend = TRUE)
dev.off()
message(sprintf("Saved clustered heatmap to %s", output_file_clustered))

# Save unclustered version
output_file_unclustered <- file.path(output_dir, "all_hearts_unclustered_heatmap.png")
png(filename = output_file_unclustered, width = 5500, height = 4000, res = 400)
draw(ht_unclustered, heatmap_legend_side = "right", annotation_legend_side = "right", merge_legend = TRUE)
dev.off()
message(sprintf("Saved unclustered heatmap to %s", output_file_unclustered))


png 
  2

Saved clustered heatmap to ./figures/heatmaps/redo_DMSO_plate/all_hearts_clustered_heatmap.png



png 
  2

Saved unclustered heatmap to ./figures/heatmaps/redo_DMSO_plate/all_hearts_unclustered_heatmap.png

